# Pre-Processing of the 2016-ANSAMO Dataset

# Pre-Processing of the 2016-ANSAMO Dataset
**Directory:**
    > `Subject_<nr>_ADL_<activity>.csv`
    > ...

**Types of Executed ADLs:**   
1) normal walking, 2) light jogging, 3) body bending, 4) hopping, 5) climbing stairs (up), 6) climbing stairs (down), 7) lying down and getting up from a bed, 8) sitting down (and up) on (from) a chair.

**Columns Units:**  
    After the header, every line in the files corresponds to a measurement captured by a particular mobility sensor of a determined node (mote or SensorTag).  
    The format of the lines, which is also explained in the file header, includes 7 numerical values separated by a semicolon:  
        -The time (in ms) since the experiment began.  
        -The number of the sample (for the same sensor and node).  
        -The three real numbers describing the measurements of the triaxial sensor (x-axis, y-axis and z-axis). The units are g, °/s or μT depending on whether the measurement was performed by an accelerometer, a gyroscope or a magnetometer, respectively.  
        -An integer (0, 1 or 2) describing the type of the sensor that originated the measurement (Accelerometer = 0 , Gyroscope = 1, Magnetometer = 2)  
        -An integer (from 0 to 4) informing about the sensing node (the correspondence between this numerical code and the Bluetooth MAC address and position of the motes is described in the file header).
        
## Desired Dataset Format

current header = TS(ms);SampleNr;X-Axis;Y-Axis;Z-Axis;SensorType;SensorID  
desired header = TS(ms); AccX; AccY; AccZ; MagnX; MagnY; MagnZ; GyroX; GyroY; GyroZ; SubjectID; Gender; Age; Position; Label; and other Feature Extraction Columns (mean, std, corr, etc...)

Pre-Processing Tasks:
    - clean headers information
    - divide the files by subject: "subject_01.csv"; "subject_02.csv"; ... with the desired header above.

In [1]:
## Readme File ##
from IPython.display import IFrame
IFrame("./2016-ANSAMO-Readme.pdf", width=800, height=800)

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
##
## first download dataset
##

import sys
sys.path.append('../../../')
from ipynb.fs.full.Utils import download_from_google_drive
download_from_google_drive('fuMxj-dnMXEPHGhHwuQoNjBovO_v5Kdk', '../../../datasets/ANSAMO-2016.zip');

The File Already Exists. Please Change The Path Destination.


In [4]:
SUBJECTS = pd.DataFrame([
    {'id': 'Subject 01', 'age':'22', 'gender':'female', 'height':'167', 'weight': '63'},
    {'id': 'Subject 02', 'age':'27', 'gender':'male', 'height':'173', 'weight': '90'},
    {'id': 'Subject 03', 'age':'23', 'gender':'male', 'height':'179', 'weight': '68'}, 
    {'id': 'Subject 04', 'age':'24', 'gender':'male', 'height':'175', 'weight': '79'},
    {'id': 'Subject 05', 'age':'28', 'gender':'male', 'height':'195', 'weight': '81'},
    
    {'id': 'Subject 06', 'age':'22', 'gender':'female', 'height':'167', 'weight': '57'},
    {'id': 'Subject 07', 'age':'55', 'gender':'male', 'height':'170', 'weight': '83'},
    {'id': 'Subject 08', 'age':'19', 'gender':'male', 'height':'178', 'weight': '68'},
    {'id': 'Subject 09', 'age':'26', 'gender':'male', 'height':'176', 'weight': '73'},
    {'id': 'Subject 10', 'age':'51', 'gender':'female', 'height':'155', 'weight': '55'},
    
    {'id': 'Subject 11', 'age':'14', 'gender':'female', 'height':'159', 'weight': '50'},
    {'id': 'Subject 12', 'age':'22', 'gender':'female', 'height':'164', 'weight': '52'},
    {'id': 'Subject 13', 'age':'26', 'gender':'male', 'height':'179', 'weight': '67'},
    {'id': 'Subject 14', 'age':'21', 'gender':'male', 'height':'173', 'weight': '77'},
    {'id': 'Subject 15', 'age':'27', 'gender':'female', 'height':'166', 'weight': '66'},
    
    {'id': 'Subject 16', 'age':'24', 'gender':'male', 'height':'177', 'weight': '66'},
    {'id': 'Subject 17', 'age':'23', 'gender':'female', 'height':'163', 'weight': '93'},
])

def read_file(filepath):
    filename = ""
    try:
        filepath.index("/");
        array = filepath.split('/');
        filename = array[len(array) - 1]
    except:
        filename = filepath
    print(filename)
    ## filename = UMAFall_Subject_01_ADL_Bending_1_2016-06-13_20-25-34.csv
    array = filename.split('_')
    subjectID = array[1] + " " + array[2]
    label = array[4]
    experiment = array[4] + "_" + array[5]
    ### read csv
    train_dataRaw = pd.read_csv(filepath)
    ### remove first 31 lines
    train_data = train_dataRaw.iloc[31:]
    ### user characteristics
    user = SUBJECTS[SUBJECTS['id'].isin([subjectID])];
    age = user['age']
    gender = user['gender']
    height = user['height']
    weight = user['weight']

    return {"filecontent": train_data, 
             'experiment': experiment, 
             'label': label, 
             'subjectID': subjectID,
             'filename': filename, 
             'gender': gender.values[0], 
             'age': age.values[0], 
             'height': height.values[0], 
             'weight': weight.values[0]};
   

###output = read_file("../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_Bending_1_2016-06-13_20-25-34.csv");

In [5]:
##  Sensor_ID	 Position	 Device Model                                   
#0	 RIGHTPOCKET	 lge-LG-H815-5.1                                
#1	 CHEST	 SensorTag
#2	 WAIST	 SensorTag
#3	 WRIST	 SensorTag                                            
#4	 ANKLE	 SensorTag                                                                                                            
RIGHTPOCKET = '0'
CHEST = '1'
WAIST = '2'
WRIST = '3'
ANKLE = '4'
#% Sensor_Type:                                                                     
#% Accelerometer = 0                                                                
#% Gyroscope = 1                                                                    
#% Magnetometer = 2 
ACCE = '0'
GYRO = '1'
MAGN = '2'
    
def split_positions_and_signals(filecontent):
    ''' filecontent = pd.DataFrame '''
    ### divide line into several columns
    header_tmp = {'ts(ms)': [], 'SampleNr': [], 'X-Axis': [], 'Y-Axis': [], 'Z-Axis': [], 'SensorType': [] , 'SensorID': []}
    values = pd.DataFrame(data=header_tmp)
    i = 0
    for index, row in filecontent.iterrows():
            line = row[0]
            array = line.split(';')
            if i > 0: values = values.append(pd.DataFrame({'ts(ms)': [array[0]], 'SampleNr': [array[1]], 'X-Axis': [array[2]], 'Y-Axis': [array[3]], 'Z-Axis': [array[4]], 'SensorType': [array[5]] , 'SensorID': [array[6]]}))
            i+=1;
    ### sorting per sensorid, sensortype, and ts
    train_data_processed_ordered = values.sort_values(['SensorID', 'SensorType', 'ts(ms)']);
    ### start splitting into positions and signals
    # divide signals per position
    train_data_pocket = train_data_processed_ordered[train_data_processed_ordered['SensorID'].isin([RIGHTPOCKET])];
    train_data_chest = train_data_processed_ordered[train_data_processed_ordered['SensorID'].isin([CHEST])];
    train_data_waist = train_data_processed_ordered[train_data_processed_ordered['SensorID'].isin([WAIST])];
    train_data_wrist = train_data_processed_ordered[train_data_processed_ordered['SensorID'].isin([WRIST])];
    train_data_ankle = train_data_processed_ordered[train_data_processed_ordered['SensorID'].isin([ANKLE])];
    # pocket 
    pocket_acc = train_data_pocket[train_data_pocket['SensorType'].isin([ACCE])];
    pocket_gyro = train_data_pocket[train_data_pocket['SensorType'].isin([GYRO])];
    pocket_magn = train_data_pocket[train_data_pocket['SensorType'].isin([MAGN])];
    # wrist
    wrist_acc = train_data_wrist[train_data_wrist['SensorType'].isin([ACCE])];
    wrist_gyro = train_data_wrist[train_data_wrist['SensorType'].isin([GYRO])];
    wrist_magn = train_data_wrist[train_data_wrist['SensorType'].isin([MAGN])];
    # ankle
    ankle_acc = train_data_ankle[train_data_ankle['SensorType'].isin([ACCE])];
    ankle_gyro = train_data_ankle[train_data_ankle['SensorType'].isin([GYRO])];
    ankle_magn = train_data_ankle[train_data_ankle['SensorType'].isin([MAGN])];
    # waist
    waist_acc = train_data_waist[train_data_waist['SensorType'].isin([ACCE])];
    waist_gyro = train_data_waist[train_data_waist['SensorType'].isin([GYRO])];
    waist_magn = train_data_waist[train_data_waist['SensorType'].isin([MAGN])];
    # chest
    chest_acc = train_data_chest[train_data_chest['SensorType'].isin([ACCE])];
    chest_gyro = train_data_chest[train_data_chest['SensorType'].isin([GYRO])];
    chest_magn = train_data_chest[train_data_chest['SensorType'].isin([MAGN])];
    
    return {'pocket_acc':pocket_acc,
           'pocket_gyro':pocket_gyro,
           'pocket_magn':pocket_magn,
           
           'wrist_acc':wrist_acc,
           'wrist_gyro':wrist_gyro,
           'wrist_magn':wrist_magn,
           
           'ankle_acc':ankle_acc,
           'ankle_gyro':ankle_gyro,
           'ankle_magn':ankle_magn,
           
           'waist_acc':waist_acc,
           'waist_gyro':waist_gyro,
           'waist_magn':waist_magn,
           
           'chest_acc':chest_acc,
           'chest_gyro':chest_gyro,
           'chest_magn':chest_magn,
           
            'train_data_pocket':train_data_pocket,
            'train_data_chest':train_data_chest,
            'train_data_waist':train_data_waist,
            'train_data_wrist':train_data_wrist,
            'train_data_ankle':train_data_ankle,
           };

##filecontent = pd.DataFrame(output['filecontent']);
##values = split_positions_and_signals(filecontent);

In [6]:
def join_all_signals_types(signals, info):
    ''' Receives the argument: 
    sigmals = pd.DataFrame([
    {   'value': pocket_acc, 'signalType': 'acc'},  {'value': pocket_gyro, 'signalType': 'gyro'}, ... 
    info = pd.DataFrame([{
        'userGender': 'male',
        'userAge': '55',
        'userID':'sdasd'
        'position': 2
        'filename': 'asda.csv',
        'experiment': 'Bending_1',
        'label': 'Bending'}]) 
    '''
    userAge = info['userAge']
    userGender = info['userGender']
    userID = info['userID']
    label = info['label']
    filename = info['filename']
    experiment = info['experiment']
    
    header = [ 'ts(ms)', 'accX', 'accY', 'accZ',
              'magX', 'magY', 'magZ',
              'gyrX', 'gyrY', 'gyrZ',
              'userGender', 'userAge', 'userID',
              'position', 
              'label',
              'filename',
              'experiment' ];
    all_values = pd.DataFrame(columns=header);
    
    for indexR, lineR in signals.iterrows():
        array = lineR['value'];
        signalType = lineR['signalType'];
        position = lineR['position'];
        
        for index, line in array.iterrows():
            row = all_values[all_values['ts(ms)'] == line['ts(ms)']];
            ts_tmp = [line['ts(ms)']];
            
            if signalType is "acc": 
                 acc_values_tmp = [ [line['X-Axis']] ,  [line['Y-Axis']],  [line['Z-Axis']]];
                 magn_values_tmp = [ [-1] ,  [-1],  [-1]];
                 gyro_values_tmp = [ [-1] ,  [-1],  [-1]]; 
            elif signalType is "magn":
                 acc_values_tmp = [ [-1] ,  [-1],  [-1]]; 
                 magn_values_tmp = [ [line['X-Axis']] ,  [line['Y-Axis']],  [line['Z-Axis']]];
                 gyro_values_tmp = [ [-1] ,  [-1],  [-1]]; 
            else:
                acc_values_tmp = [ [-1] ,  [-1],  [-1]];    
                magn_values_tmp = [ [-1] ,  [-1],  [-1]];
                gyro_values_tmp = [ [line['X-Axis']] ,  [line['Y-Axis']],  [line['Z-Axis']]];      

            if row.size == 0:
                all_values = all_values.append(pd.DataFrame({'ts(ms)': ts_tmp, 
                                    'accX':acc_values_tmp[0],
                                    'accY':acc_values_tmp[1],
                                    'accZ': acc_values_tmp[2], 
                                    'magX':magn_values_tmp[0],
                                    'magY':magn_values_tmp[1], 
                                    'magZ': magn_values_tmp[2], 
                                    'gyrX':gyro_values_tmp[0],
                                    'gyrY': gyro_values_tmp[1],
                                    'gyrZ': gyro_values_tmp[2], 
                                    'userGender': userGender, 
                                    'userAge': userAge, 
                                    'userID': userID,
                                    'position': position, 
                                    'label':label.values,
                                    'filename':filename,
                                    'experiment': experiment}))
            else:
                row.at[index, 'accX'] = acc_values_tmp[0];
                row.at[index, 'accY'] = acc_values_tmp[1];
                row.at[index, 'accZ'] = acc_values_tmp[2];
                
                row.at[index, 'gyrX'] = gyro_values_tmp[0];
                row.at[index, 'gyrY'] = gyro_values_tmp[1];
                row.at[index, 'gyrZ'] = gyro_values_tmp[2];
                
                row.at[index, 'magZ'] = magn_values_tmp[2];
                row.at[index, 'magZ'] = magn_values_tmp[2];
                row.at[index, 'magZ'] = magn_values_tmp[2];
                
    return all_values;

###
###signals = pd.DataFrame([{'value': wrist_gyro, 'signalType': 'gyro'}])
###all_values = join_all_signals_types(signals)
###print("len of all values:", len(all_values));

In [7]:
def process_file(filePath):
    ### read file
    output = read_file(filePath)
    filecontent = pd.DataFrame(output['filecontent']);
    ###
    experiment = output['experiment'];
    label = output['label'];
    subjectID = output['subjectID'];
    filename = output['filename'];
    gender = output['gender'];
    age = output['age'];
    height = output['height'];
    weight = output['weight'];
    info = pd.DataFrame([{
        'userGender': gender,
        'userAge': age,
        'userID': subjectID,
        'filename': filename,
        'experiment': experiment,
        'label': label
    }])
    ### split into positions and signal type
    values = split_positions_and_signals(filecontent);
    ### 
    pocket_acc = values['pocket_acc']
    pocket_gyro = values['pocket_gyro']
    pocket_magn = values['pocket_magn']

    wrist_acc = values['wrist_acc']
    wrist_gyro = values['wrist_gyro']
    wrist_magn = values['wrist_magn']

    ankle_acc = values['ankle_acc']
    ankle_gyro = values['ankle_gyro']
    ankle_magn = values['ankle_magn']

    waist_acc = values['waist_acc']
    waist_gyro = values['waist_gyro']
    waist_magn = values['waist_magn']

    chest_acc = values['chest_acc']
    chest_gyro = values['chest_gyro']
    chest_magn = values['chest_magn']

    signalsPocket = pd.DataFrame(
        [
         {'value': pocket_acc, 'signalType': 'acc', 'position': 'pocket'},
         {'value': pocket_gyro, 'signalType': 'gyro', 'position': 'pocket'},
         {'value': pocket_magn, 'signalType': 'magn', 'position': 'pocket'},
        ]);

    signalsAnkle = pd.DataFrame(
        [
        {'value': ankle_acc, 'signalType': 'acc', 'position': 'ankle'},
        {'value': ankle_gyro, 'signalType': 'gyro', 'position': 'ankle'},
        {'value': ankle_magn, 'signalType': 'magn', 'position': 'ankle'},
        ]);

    signalsWrist = pd.DataFrame(
        [
         {'value': wrist_acc, 'signalType': 'acc', 'position': 'wrist'},
         {'value': wrist_gyro, 'signalType': 'gyro', 'position': 'wrist'},
         {'value': wrist_magn, 'signalType': 'magn', 'position': 'wrist'},
        ]);

    signalsWaist = pd.DataFrame(
        [
        {'value': waist_acc, 'signalType': 'acc', 'position': 'waist'},
        {'value': waist_gyro, 'signalType': 'gyro', 'position': 'waist'},
        {'value': waist_magn, 'signalType': 'magn', 'position': 'waist'},
        ]);

    signalsChest = pd.DataFrame(
        [
        {'value': chest_acc, 'signalType': 'acc', 'position': 'chest'},
        {'value': chest_gyro, 'signalType': 'gyro', 'position': 'chest'},
        {'value': chest_magn, 'signalType': 'magn', 'position': 'chest'},
        ]);
    ### join all values
    all_values = join_all_signals_types(signalsPocket, info);
    all_values = all_values.append(join_all_signals_types(signalsAnkle, info))
    all_values = all_values.append(join_all_signals_types(signalsWrist, info))
    all_values = all_values.append(join_all_signals_types(signalsWaist, info))
    all_values = all_values.append(join_all_signals_types(signalsChest, info))
    
    return all_values;

In [9]:
# write to csv
import os
base_path = '../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset'
files = os.listdir(base_path)
total = pd.DataFrame();
for file in files:
    if '.csv' in file:
        print("processing file: " + base_path + "/" + file + " wait...")
        filePath = base_path+"/"+file;
        values = process_file(filePath);
        total = total.append(values)

total.to_csv("all_values.csv", sep=',')
print("done!")

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_Bending_1_2016-06-13_20-25-34.csv wait...
UMAFall_Subject_01_ADL_Bending_1_2016-06-13_20-25-34.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_Bending_2_2016-06-13_20-26-15.csv wait...
UMAFall_Subject_01_ADL_Bending_2_2016-06-13_20-26-15.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_Bending_3_2016-06-13_20-26-59.csv wait...
UMAFall_Subject_01_ADL_Bending_3_2016-06-13_20-26-59.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_forwardFall_1_2016-06-13_20-43-52.csv wait...
UMAFall_Subject_01_ADL_forwardFall_1_2016-06-13_20-43-52.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_01_ADL_Hopping_1_2016-06-13_20-37-40.csv wait...
UMAFall_Subject_01_ADL_Hopping_1_2016-06-13_20-37-40.csv
processing file: ../../../datasets/ANSAMO-

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_02_ADL_Hopping_2_2016-06-13_13-01-42.csv wait...
UMAFall_Subject_02_ADL_Hopping_2_2016-06-13_13-01-42.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_02_ADL_Hopping_3_2016-06-13_13-02-19.csv wait...
UMAFall_Subject_02_ADL_Hopping_3_2016-06-13_13-02-19.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_02_ADL_Jogging_1_2016-06-13_13-07-16.csv wait...
UMAFall_Subject_02_ADL_Jogging_1_2016-06-13_13-07-16.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_02_ADL_Jogging_2_2016-06-13_13-07-58.csv wait...
UMAFall_Subject_02_ADL_Jogging_2_2016-06-13_13-07-58.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_02_ADL_Jogging_3_2016-06-13_13-08-41.csv wait...
UMAFall_Subject_02_ADL_Jogging_3_2016-06-13_13-08-41.csv
processing file: ../../../datasets/ANSAMO-2016/UMA

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_03_ADL_Sitting_GettingUpOnAChair_3_2016-06-15_21-51-18.csv wait...
UMAFall_Subject_03_ADL_Sitting_GettingUpOnAChair_3_2016-06-15_21-51-18.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_03_ADL_Walking_1_2016-06-15_21-43-41.csv wait...
UMAFall_Subject_03_ADL_Walking_1_2016-06-15_21-43-41.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_03_ADL_Walking_2_2016-06-15_21-44-20.csv wait...
UMAFall_Subject_03_ADL_Walking_2_2016-06-15_21-44-20.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_03_ADL_Walking_3_2016-06-15_21-44-58.csv wait...
UMAFall_Subject_03_ADL_Walking_3_2016-06-15_21-44-58.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_03_Fall_backwardFall_1_2016-06-15_21-30-13.csv wait...
UMAFall_Subject_03_Fall_backwardFall_1_2016-06-15_21-30-13.csv
pr

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_04_ADL_Walking_2_2016-06-13_21-12-36.csv wait...
UMAFall_Subject_04_ADL_Walking_2_2016-06-13_21-12-36.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_04_ADL_Walking_3_2016-06-13_21-13-22.csv wait...
UMAFall_Subject_04_ADL_Walking_3_2016-06-13_21-13-22.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_04_Fall_backwardFall_1_2016-06-13_21-45-05.csv wait...
UMAFall_Subject_04_Fall_backwardFall_1_2016-06-13_21-45-05.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_04_Fall_backwardFall_2_2016-06-13_21-45-45.csv wait...
UMAFall_Subject_04_Fall_backwardFall_2_2016-06-13_21-45-45.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_04_Fall_backwardFall_3_2016-06-13_21-46-29.csv wait...
UMAFall_Subject_04_Fall_backwardFall_3_2016-06-13_21-46-29.csv
processing fil

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_06_ADL_Sitting_GettingUpOnAChair_1_2016-06-18_14-25-19.csv wait...
UMAFall_Subject_06_ADL_Sitting_GettingUpOnAChair_1_2016-06-18_14-25-19.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_06_ADL_Sitting_GettingUpOnAChair_2_2016-06-18_14-25-59.csv wait...
UMAFall_Subject_06_ADL_Sitting_GettingUpOnAChair_2_2016-06-18_14-25-59.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_06_ADL_Walking_1_2016-06-18_14-24-36.csv wait...
UMAFall_Subject_06_ADL_Walking_1_2016-06-18_14-24-36.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_06_Fall_backwardFall_1_2016-06-18_14-31-39.csv wait...
UMAFall_Subject_06_Fall_backwardFall_1_2016-06-18_14-31-39.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_06_Fall_backwardFall_2_2016-06-18_14-32-18.csv wait...
UMAFall_Subject_0

...

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_13_Fall_backwardFall_2_2016-06-17_18-44-01.csv wait...
UMAFall_Subject_13_Fall_backwardFall_2_2016-06-17_18-44-01.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_13_Fall_backwardFall_3_2016-06-17_18-43-19.csv wait...
UMAFall_Subject_13_Fall_backwardFall_3_2016-06-17_18-43-19.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_13_Fall_backwardFall_3_2016-06-17_18-44-43.csv wait...
UMAFall_Subject_13_Fall_backwardFall_3_2016-06-17_18-44-43.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_13_Fall_backwardFall_4_2016-06-17_18-45-23.csv wait...
UMAFall_Subject_13_Fall_backwardFall_4_2016-06-17_18-45-23.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_13_Fall_forwardFall_1_2016-06-17_18-38-53.csv wait...
UMAFall_Subject_13_Fall_forwardFall_1_2016-06-17_18-38

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_15_Fall_lateralFall_2_2016-06-17_19-23-53.csv wait...
UMAFall_Subject_15_Fall_lateralFall_2_2016-06-17_19-23-53.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_15_Fall_lateralFall_3_2016-06-17_19-24-31.csv wait...
UMAFall_Subject_15_Fall_lateralFall_3_2016-06-17_19-24-31.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_15_Fall_lateralFall_4_2016-06-17_19-28-33.csv wait...
UMAFall_Subject_15_Fall_lateralFall_4_2016-06-17_19-28-33.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_Bending_1_2016-05-29_20-34-24.csv wait...
UMAFall_Subject_16_ADL_Bending_1_2016-05-29_20-34-24.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_Bending_2_2016-05-29_20-36-13.csv wait...
UMAFall_Subject_16_ADL_Bending_2_2016-05-29_20-36-13.csv
processing file: ../

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_LyingDown_OnABed_3_2016-05-29_20-53-15.csv wait...
UMAFall_Subject_16_ADL_LyingDown_OnABed_3_2016-05-29_20-53-15.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_LyingDown_OnABed_4_2016-05-31_18-47-14.csv wait...
UMAFall_Subject_16_ADL_LyingDown_OnABed_4_2016-05-31_18-47-14.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_LyingDown_OnABed_5_2016-05-31_18-48-12.csv wait...
UMAFall_Subject_16_ADL_LyingDown_OnABed_5_2016-05-31_18-48-12.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_LyingDown_OnABed_6_2016-05-31_18-48-56.csv wait...
UMAFall_Subject_16_ADL_LyingDown_OnABed_6_2016-05-31_18-48-56.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_ADL_LyingDown_OnABed_7_2016-06-02_00-07-30.csv wait...
UMAFall_Subject_16_ADL_Lyi

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_Fall_backwardFall_7_2016-06-01_22-51-58.csv wait...
UMAFall_Subject_16_Fall_backwardFall_7_2016-06-01_22-51-58.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_Fall_backwardFall_8_2016-06-01_22-53-56.csv wait...
UMAFall_Subject_16_Fall_backwardFall_8_2016-06-01_22-53-56.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_Fall_backwardFall_9_2016-06-01_22-54-59.csv wait...
UMAFall_Subject_16_Fall_backwardFall_9_2016-06-01_22-54-59.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_Fall_forwardFall_10_2016-06-01_22-14-39.csv wait...
UMAFall_Subject_16_Fall_forwardFall_10_2016-06-01_22-14-39.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_16_Fall_forwardFall_11_2016-06-01_22-18-31.csv wait...
UMAFall_Subject_16_Fall_forwardFall_11_2016-06-01_22-

processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_17_ADL_Bending_2_2016-06-07_21-04-19.csv wait...
UMAFall_Subject_17_ADL_Bending_2_2016-06-07_21-04-19.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_17_ADL_Bending_3_2016-06-07_21-05-02.csv wait...
UMAFall_Subject_17_ADL_Bending_3_2016-06-07_21-05-02.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_17_ADL_Hopping_1_2016-06-07_21-17-08.csv wait...
UMAFall_Subject_17_ADL_Hopping_1_2016-06-07_21-17-08.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_17_ADL_Hopping_2_2016-06-07_21-17-53.csv wait...
UMAFall_Subject_17_ADL_Hopping_2_2016-06-07_21-17-53.csv
processing file: ../../../datasets/ANSAMO-2016/UMA_ADL_FALL_Dataset/UMAFall_Subject_17_ADL_Hopping_3_2016-06-07_21-18-38.csv wait...
UMAFall_Subject_17_ADL_Hopping_3_2016-06-07_21-18-38.csv
processing file: ../../../datasets/ANSAMO-2016/UMA

In [25]:
### Done